### Naive implementation of linear regression

In [1]:
%matplotlib inline
import random
import torch

- synthetic data set

In [2]:
def synthetic_data(w, b, nums):
    # 生成一个形状为 (nums, len(w)) 的张量 X，
    # 每个元素都是从均值为 0 和标准差为 1 的正态分布中采样的随机数
    X = torch.normal(0, 1, (nums, len(w)))
    y = torch.matmul(X, w) + b

    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

real_w = torch.tensor([4.0, 8])
real_b = 13

features, labels = synthetic_data(real_w, real_b, 1000)

- Read data set

In [3]:
def data_iter(batch_size, features, labels):
    nums = len(features)
    # 随机读取样本
    index = list(range(nums))
    random.shuffle(index)
    for i in range(0, nums, batch_size):
        batch_index = torch.tensor(
            index[i : min(i + batch_size, nums)]
        )
        yield features[batch_index], labels[batch_index]
        
# babatch_size = 10
# for X, y in data_iter(batch_size, features, labels):
#     print(X, '\n', y)
#     break

- Definition model

In [4]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

- Loss Function

In [5]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

- Define optimization algorithm

In [6]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

- trainning

In [7]:
# 初始化随机参数
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
# 超参数
batch_size = 10
lr = 0.02
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward();
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        # 打印当前轮次和平均损失，损失值格式化为浮点数
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 2.320065
epoch 2, loss 0.044475
epoch 3, loss 0.000910


In [10]:
print(w)
print(b)
print(f'w的估计误差: {real_w - w.reshape(real_w.shape)}')
print(f'b的估计误差: {real_b - b}')

tensor([[3.9932],
        [7.9757]], requires_grad=True)
tensor([12.9666], requires_grad=True)
w的估计误差: tensor([0.0068, 0.0243], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0334], grad_fn=<RsubBackward1>)
